# Assignment 4: Ranking & Evaluation

## General guidelines

This notebook contains considerable amount of code to help you complete this assignment. Your task is to implement any missing parts of the code and answer any questions (if exist) within this notebook. This will require understanding the existing code, may require reading about packages being used, reading additional resources, and maybe even going over your notes from class 😱

**Evaluation and auto-grading**: Your submissions will be evaluated using both automatic and manual grading. Code parts for implementation are marked with a comment `# YOUR CODE HERE`, and usually followed by cell(s) containing automatic tests that evaluate the correctness of your answer. Additionaly, staff will manually assess your submission. Any automatic tests that did not run due to your notebook timing out **will automatically receive 0 points**. The execution time excludes initial data download, which will already exist in the testing environment. The staff reserves the right to **modify any grade provided by the auto-grader** as well as to **execute additional tests not provided to you**. It is also important to note that **auto-graded cells only result in full or no credit**. In other words, you must pass all tests implemented in a test cell in order to get the credit for it, and passing some, but not all, of the tests in a test cell will not earn you any points for that cell. 

**Submission**: Unless specified otherwise, you need to upload this notebook file **with your ID as the file name**, e.g. 012345678.ipynb, to the assignment on Moodle.

# Tasks

In this assignment, we are going to build a complete retrieval system using the inverted index (assignment 2)

**Your tasks in this assignment are:**

1. (40 Points) Implement ranking using TF-IDF and BM25.
2. (35 Points) Implement search/retrieval using the inverted index. Implement compound ranking based on title and body. 
3. (25 Points) Using weighting of title and body scores. Provide three examples of mistakes that the model is making and explanations for why, and describe how you will change the model based on these observations.



# Imports

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict,Counter
import re
import nltk
import pickle
import numpy as np
nltk.download('stopwords')

from nltk.corpus import stopwords
from tqdm import tqdm
import operator
from itertools import islice,count
from contextlib import closing

import json
from io import StringIO
from pathlib import Path
from operator import itemgetter
import pickle
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tasks 1: Implement ranking using TF-IDF and BM25 (40 points).
In this task, you need to implement TF-IDF and BM25. At this point, for simplicity, we do not use an inverted index. However, later on in this assignment, we will.

Implementation remarks:
* TF-IDF: use [sklearn TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). To deal with stopwords use the argument `stop_words`. (5 points)

TfidfVectorizer suggests handling with additional parameters, as you can read in the documentation. Make sure you read about them.

* Cosine Similarity: use [sklearn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html). (10 points)

* BM25: Implement a BM25 version according to the provided skeleton **without the use of packages**. 15 points as follows:
    * Prepare the data and filter stopwords. (5 points)
    * Implement two functions at BM25 class. (10 points)

* Ranking: implement topN functionallity (10 points)

**Later in this assignment, we will write code for TF-IDF and BM25 that utilize the inverted index.**

In [ ]:
# set of documents
data = ['The sky is blue and we can see the blue sun.',
        'The sun is bright and yellow.',
        'here comes the blue sun',
        'Lucy in the sky with diamonds and you can see the sun in the sky',
        'sun sun blue sun here we come',
        'Lucy likes blue bright diamonds']

### TF-IDF

**YOUR TASK (5 POINTS):**  Complete the implementation of `tf_idf_scores`, which calculates the tfidf for each word in a single document utilizing TfidfVectorizer via sklearn.

In [ ]:
def tf_idf_scores(data):
    """
    This function calculates the tfidf for each word in a single document utilizing TfidfVectorizer via sklearn.

    Parameters:
    -----------
      data: list of strings.
    
    Returns:
    --------
      Two objects as follows:
                                a) DataFrame, documents as rows (i.e., 0,1,2,3, etc'), terms as columns ('bird','bright', etc').
                                b) TfidfVectorizer object.

    """
    # YOUR CODE HERE
    vectorizer = TfidfVectorizer(input = data, stop_words=str("english"))
    df = pd.DataFrame({'docId' : [i for i in range(len(data))], 'texts' : data})
    vect_sp = vectorizer.fit_transform(df['texts'])
    vect = pd.DataFrame.sparse.from_spmatrix(vect_sp, columns=vectorizer.get_feature_names_out())
    return vect, vectorizer
    
df_tfidfvect, tfidfvectorizer = tf_idf_scores(data)

In [ ]:
#tests
assert df_tfidfvect.shape[1] == 10 and df_tfidfvect.shape[0] == 6
assert 'is' not in df_tfidfvect.columns and 'we' not in df_tfidfvect.columns
assert 'sun' in df_tfidfvect.columns and 'yellow' in df_tfidfvect.columns
assert round(df_tfidfvect.max(),3).max() == 0.798
assert np.count_nonzero(df_tfidfvect) == 21
assert type(tfidfvectorizer) == TfidfVectorizer

Now, upon existing TF-IDF matrix we are seeking for the similarity for given new queries. 
First we need to convert the **new** queries into a vector format. Therefore we use transform instead of fit_transform.

Next, we would like to calculate the cosine similarity between queires and documents.



In [ ]:
queries = ['look the the blue sky', 'He likes the blue the sun','Lucy likes blue sky with diamonds']
queries_vector = tfidfvectorizer.transform(queries)

Now, lets calculate the cosine similarity utilizing sklearn.

**YOUR TASK (10 POINTS):**  Complete the implementation of `cosine_sim_using_sklearn`.
You need to compute the similarity between the queries and the given documents.

In [ ]:
def cosine_sim_using_sklearn(queries,tfidf):
    """
    In this function you need to utilize the cosine_similarity function from sklearn.
    You need to compute the similarity between the queries and the given documents.
    This function will return a DataFrame in the following shape: (# of queries, # of documents).
    Each value in the DataFrame will represent the cosine_similarity between given query and document.
    
    Parameters:
    -----------
      queries: sparse matrix represent the queries after transformation of tfidfvectorizer.
      documents: sparse matrix represent the documents.
      
    Returns:
    --------
      DataFrame: This function will return a DataFrame in the following shape: (# of queries, # of documents).
      Each value in the DataFrame will represent the cosine_similarity between given query and document.
    """
    return pd.DataFrame(cosine_similarity(queries, tfidf))

cosine_sim_df = cosine_sim_using_sklearn(queries_vector,df_tfidfvect)

In [ ]:
# tests for cosine similarity
assert cosine_sim_df.shape[0] == len(queries)
assert cosine_sim_df.shape[1] == len(data)
assert (abs(cosine_sim_df)>1).any().any() == False
assert np.count_nonzero(cosine_sim_df) == 16
assert round(cosine_sim_df.max(),3).max() == 0.888

### BM25
In this section we will create a class of Best Match 25 (BM25)

To use BM25 we will need to following parameters:

* $k1$ and $b$ - free parameters
* $f(t_i,D)$ - term frequency of term $t_i$ in document $D$
* |$D$|- is the length of the document $D$ in terms 
* $avgdl$ -  average document length
* $IDF$ - which is calculted as follows: $ln(\frac{(N-n(t_i)+0.5}{n(t_i)+0.5)}+1)$, where $N$ is the total number of documents in the collection, and $n(t_i)$ is the number of documents containing $t_i$ (e.g., document frequency (df)).

As a reminder of the data looks like this


In [ ]:
data

['The sky is blue and we can see the blue sun.',
 'The sun is bright and yellow.',
 'here comes the blue sun',
 'Lucy in the sky with diamonds and you can see the sun in the sky',
 'sun sun blue sun here we come',
 'Lucy likes blue bright diamonds']

**Note:** While using TF-IDF from sklearn we used the `stop_words` argument. 

**It is not the case when working with BM25 without any package.**
Therefore, we need to filter the data and clean it. We will do so utilizing NLTK stopwords

In [ ]:
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stopwords_frozen = frozenset(stopwords.words('english'))
def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in stopwords_frozen]    
    return list_of_tokens


clean_data = [tokenize(doc) for doc in data]
clean_data

[['sky', 'blue', 'see', 'blue', 'sun'],
 ['sun', 'bright', 'yellow'],
 ['comes', 'blue', 'sun'],
 ['lucy', 'sky', 'diamonds', 'see', 'sun', 'sky'],
 ['sun', 'sun', 'blue', 'sun', 'come'],
 ['lucy', 'likes', 'blue', 'bright', 'diamonds']]

Now let's find all the parameters needed for our toy example.
**Later on we will need to gather this information from the inverted index.**

**YOUR TASK (5 POINTS):**  Complete the implementation of `bm25_preprocess`.
This function goes through the data and saves relevant information for the calculation of bm25. 

In [ ]:
def bm25_preprocess(data):
    """
    This function goes through the data and saves relevant information for the calculation of bm25. 
    Specifically, in this function, we will create 3 objects that gather information regarding document length, term frequency and
    document frequency.
    Parameters
    -----------
    data: list of lists. Each inner list is a list of tokens. 
    Example of data: 
    [
        ['sky', 'blue', 'see', 'blue', 'sun'],
        ['sun', 'bright', 'yellow'],
        ['comes', 'blue', 'sun'],
        ['lucy', 'sky', 'diamonds', 'see', 'sun', 'sky'],
        ['sun', 'sun', 'blue', 'sun'],
        ['lucy', 'likes', 'blue', 'bright', 'diamonds']
    ]
    
    Returns:
    -----------
    three objects as follows:
                a) doc_len: list of integer. Each element represents the length of a document.
                b) tf: list of dictionaries. Each dictionary corresponds to a document as follows:
                                                                    key: term
                                                                    value: normalized term frequency (by the length of document)

                                                                                               
                c) df: dictionary representing the document frequency as follows:
                                                                    key: term
                                                                    value: document frequency
    """      
    doc_len = []
    tf = []
    df = {}
    words = []
    for row in data:
      doc_len.append(len(row))
      nums = Counter(row)
      new_dict = {}
      uniques = []
      for tok in row:
        new_dict[tok] = nums[tok]/len(row)
        if tok not in uniques:
          uniques.append(tok)
          words.append(tok)
      tf.append(new_dict)   
    # words = list(set(words))
    df = Counter(words)

    return doc_len,tf,df

In [ ]:
# tests - preprocess data for naive bm25
doc_len,tf,df = bm25_preprocess(clean_data)
assert df['blue'] == 4
assert sum(tf[0].values()) == 1
assert sum(doc_len) > sum(df.values())

**YOUR TASK (10 POINTS):** Complete the implementation of `calc_idf` and `_score` at the BM25 class.


In [ ]:
import math

class BM25:
    """
    Best Match 25.

    Parameters to tune
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    Attributes
    ----------
    tf_ : list[dict[str, int]]
        Term Frequency per document. So [{'hi': 1}] means
        the first document contains the term 'hi' 1 time.
        The frequnecy is normilzied by the max term frequency for each document.

    doc_len_ : list[int]
        Number of terms per document. So [3] means the first
        document contains 3 terms. 
        
    df_ : dict[str, int]
        Document Frequency per term. i.e. Number of documents in the
        corpus that contains the term.       

    avg_doc_len_ : float
        Average number of terms for documents in the corpus.

    idf_ : dict[str, float]
        Inverse Document Frequency per term.
    """

    def __init__(self,doc_len,df,tf=None,k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1
        self.tf_ = tf
        self.doc_len_ = doc_len
        self.df_ = df
        self.N_ = len(doc_len)
        self.avgdl_ = sum(doc_len) / len(doc_len)        
        

    def calc_idf(self,query):
        """
        This function calculate the idf values according to the BM25 idf formula for each term in the query.
        
        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        
        Returns:
        -----------
        idf: dictionary of idf scores. As follows: 
                                                    key: term
                                                    value: bm25 idf score
        """
        idf = {}        
        for term in query:            
          n_ti = self.df_[term]
          idf[term] = math.log(1 + (self.N_ - n_ti + 0.5) / (n_ti + 0.5))                           
        return idf
        

    def search(self, queries):
        """
        This function use the _score function to calculate the bm25 score for all queries provided.
        
        Parameters:
        -----------
        queries: list of lists. Each inner list is a list of tokens. For example:
                                                                                    [
                                                                                        ['look', 'blue', 'sky'],
                                                                                        ['likes', 'blue', 'sun'],
                                                                                        ['likes', 'diamonds']
                                                                                    ]

        Returns:
        -----------
        list of scores of bm25
        """
        scores = []
        for query in queries:            
            scores.append([self._score(query, doc_id) for doc_id in range(self.N_)])
        return scores

In [ ]:
class BM25(BM25):

    def _score(self, query, doc_id):
        """
        This function calculate the bm25 score for given query and document.
        
        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.
        
        Returns:
        -----------
        score: float, bm25 score.
        """
        #####################################################################################################
        score = 0.0             
        i = doc_id
        B = 1-self.b+self.b*(self.doc_len_[i]/self.avgdl_)
        for j in query:
          if j in self.tf_[i]:
            tfij=((self.k1+1)*self.tf_[i][j])/(B*self.k1+self.tf_[i][j])
            # idf=math.log((self.N_+1)/self.df_[j])
            # we work on a single query so screw the tfiq
            score += tfij*self.calc_idf(query)[j]
        return score
        #########################################################################################################

Let's create a new instance of BM25, and calculte its score for all queries. 
Pay attetnion - we need to tokenize and filter the stopwords from the original queries.

In [ ]:
bm25 = BM25(tf=tf,doc_len=doc_len,df=df)

# Remove tokenizing and remove stopwords from queries
clean_queries = [tokenize(query) for query in queries]
BM25_res = bm25.search(clean_queries)
BM25_df = pd.DataFrame(data = BM25_res,index = [query_id for query_id in range(len(clean_queries))] ,columns = [doc_id for doc_id in range(len(data))])

In [ ]:
# tests BM25
assert BM25_df[0][0] == BM25_df[0][2]
assert BM25_df[2][0] == BM25_df[2][2]
assert BM25_df[3][0] != BM25_df[3][2]
assert (BM25_df>1).any().any() == True
assert (BM25_df==0).any().any() == True
assert BM25_df.sum().argmax() == 5
assert BM25_df.sum().argmin() == 1
assert sum(bm25.search([['amit','livne'],['lucy','likes','blue','bright','diamonds']])[0]) == 0
assert (bm25.search([['amit','livne'],['lucy','likes','blue','bright','diamonds']])[1][-1] > BM25_df[5][2])
assert (bm25.search([['amit','livne'],['lucy','likes','blue','bright','diamonds']])[1][2] > BM25_df[1][2])

We can now search for top-N documents for each query

**YOUR TASK (10 POINTS):**  Complete the implementation of `top_N_documents`, which sort and filter the top N docuemnts (by score) for each query.


In [ ]:
def top_N_documents(df,N):
    """
    This function sort and filter the top N docuemnts (by score) for each query.
    
    Parameters
    ----------    
    df: DataFrame (queries as rows, documents as columns)
    N: Integer (how many document to retrieve for each query)    

    Returns:
    ----------
    top_N: dictionary is the following stracture:
          key - query id.
          value - sorted (according to score) list of pairs lengh of N. Eac pair within the list provide the following information (doc id, score)
    """    
    top_dict = {}
    for i in range(df.shape[0]):
      all_list = []
      for j in range(df.shape[0]):
        all_list.append((i,df.iloc[i,j]))
      all_list.sort(reverse=True)
      top_list = all_list[:N]
      top_dict[i] = top_list
    return top_dict


In [ ]:
# test - topN documents
top_2_docs = top_N_documents(cosine_sim_df,2)
top_10_docs = top_N_documents(cosine_sim_df,10)
assert len(top_2_docs[0]) == 2
assert len(top_2_docs[0][0]) == 2
assert (top_2_docs[0][0][1] > top_N_documents(cosine_sim_df,2)[0][1][1])
assert (top_10_docs[0][-1][-1] == 0)
assert (top_10_docs[0][-1][-1] != top_10_docs[1][-1][-1])
assert (top_10_docs[0][0][-1] != top_10_docs[1][0][-1])

The top-N functionality runs on all document and create a large DataFrame. What will happen if the number of documents is much larger? 
Will this function be efficient? 

Next, we will build an inverted index (like we did in assignment two). Then, we will use the inverted index utilizing the posting list to create a candidate list of documents per query. 
Therefore, we will narrow the documents relevant per query.

Only then will we calculate the TF-IDF and cosine similarity.

## Tasks 2: Implement compound ranking based on title and body using the inverted index (35 points).

In order to do so, we will first perform parsing followed by building an Inverted index.
Then we will create a searching and ranking mechanism.

In this task, you will use Cranfield Corpus. But we need to do some setups in advance. 

**Setups:**

*Make sure you upload the carnfield.py file. attached with this assignment*

First, we will load it into the following data structures:

* cran_txt_data - list of tuples. An example is provided below. Hold information regards the carnfield dataset.
* cran_qry_rel_data - list of tuples. An example is provided below. Holds information regards the queries.

We have already done all the load and preprocessing of the data for you.

We split the queries data into the training and test sets.
Next, we build two inverted indices (one built upon title information and the second on the text\body of the documents). 

**After finishing the setups, we implement the ranking mechanism (TF-IDF and BM25 build upon inverted index).**











### Cranfield Corpus

You can get the corpus from [this link](http://ir.dcs.gla.ac.uk/resources/test_collections/cran/).  <br>


**Parsing**

For detailed information about the parsing of this corpus look at [ this Notebook](https://colab.research.google.com/github/pragmalingu/experiments/blob/master/00_Data/CranfieldCorpus.ipynb) or, for parsing in general, read [this guide](https://pragmalingu.de/docs/guides/how-to-parse). An overview of the format of the files can be found here: [Data Sets](https://pragmalingu.de/docs/guides/data-comparison)

In [ ]:
!wget http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
!tar -xf cran.tar.gz
import carnfield

cran_txt_data,cran_qry_rel_data = carnfield.main()

--2022-12-20 15:26:55--  http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
Resolving ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)... 130.209.240.253
Connecting to ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)|130.209.240.253|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 506960 (495K) [application/gzip]
Saving to: ‘cran.tar.gz.1’

cran.tar.gz.1       100%[===================>] 495.08K  --.-KB/s    in 0.1s    

2022-12-20 15:26:55 (4.61 MB/s) - ‘cran.tar.gz.1’ saved [506960/506960]



### Data seperation
In real-world scenarios, we do not know all the queries in advance. Thus, we are familiar with some queries and can use them to tune the parameters of our model (e.g., the k and b for BM25 or term that appears at least n times within the data, etc.'). We will split the queries into two sets to simulate this scenario: Train and Test.

You can and should use the data from the train set to tune the parameters of your retrieval model. **You should not use the test data for tuning.**


The test data should be used to evaluate your model according to the metrics in the following section (e.g., precision, recall etc.')

Example of cran_txt_data:

In [ ]:
list(islice(cran_txt_data.items(), 1))

[('1',
  {'title': ' experimental investigation of the aerodynamics of a wing in a slipstream . ',
   'author': ' brenckman,m. ',
   'publication_date': ' j. ae. scs. 25, 1958, 324. ',
   'text': '  an experimental study of a wing in a propeller slipstream was made in order to determine the spanwise distribution of the lift increase due to slipstream at different angles of attack of the wing and at different free stream to slipstream velocity ratios .  the results were intended in part as an evaluation basis for different theoretical treatments of this problem .   the comparative span loading curves, together with supporting evidence, showed that a substantial part of the lift increment produced by the slipstream was due to a /destalling/ or boundary-layer-control effect .  the integrated remaining lift increment, after subtracting this destalling lift, was found to agree well with a potential flow theory .   an empirical evaluation of the destalling effects was made for the specific c

Example of cran_qry_rel_data. 
For each query id you can see the question and the relevance assessments is a list of tuples of relevant document. The tuple structure is as follows: (document id, relevance score). Therefore a tuple such as (184,2) indicates that the document id 184 relevance score is 2.

In [ ]:
list(islice(cran_qry_rel_data.items(), 1))

[(1,
  {'question': ' what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . ',
   'relevance_assessments': [(184, 2),
    (29, 2),
    (31, 2),
    (12, 3),
    (51, 3),
    (102, 3),
    (13, 4),
    (14, 4),
    (15, 4),
    (57, 2),
    (378, 2),
    (859, 2),
    (185, 3),
    (30, 3),
    (37, 3),
    (52, 4),
    (142, 4),
    (195, 4),
    (875, 2),
    (56, 3),
    (66, 3),
    (95, 3),
    (462, 4),
    (497, 3),
    (858, 3),
    (876, 3),
    (879, 3),
    (880, 3),
    (486, 1)]})]

Split to train and test queries

In [ ]:
cran_qry_rel_data_train = list(islice(cran_qry_rel_data.items(), 0,180))
cran_qry_rel_data_test = list(islice(cran_qry_rel_data.items(), 180,225))

Next, we will use the inverted index from assignment 2 (with some minor adaptations).
We save a dictionary named `DL`, which fetches the document length of each document.

Upon using the index functionality of add_doc, `DL` needs to be updated as well.

### Inverted index (code from assignment 2)

#### Helper classes (code from assignment 2)

In [ ]:
# Let's start with a small block size of 30 bytes just to test things out. 
BLOCK_SIZE = 30

class MultiFileWriter:
    """ Sequential binary writer to multiple files of up to BLOCK_SIZE each. """
    def __init__(self, base_dir, name):
        self._base_dir = Path(base_dir)
        self._name = name
        self._file_gen = (open(self._base_dir / f'{name}_{i:03}.bin', 'wb') 
                          for i in count())
        self._f = next(self._file_gen)
    
    def write(self, b):
      locs = []
      while len(b) > 0:
        pos = self._f.tell()
        remaining = BLOCK_SIZE - pos
        # if the current file is full, close and open a new one.
        if remaining == 0:  
          self._f.close()
          self._f = next(self._file_gen)
          pos, remaining = 0, BLOCK_SIZE
        self._f.write(b[:remaining])
        locs.append((self._f.name, pos))
        b = b[remaining:]
      return locs

    def close(self):
      self._f.close()

class MultiFileReader:
  """ Sequential binary reader of multiple files of up to BLOCK_SIZE each. """
  def __init__(self):
    self._open_files = {}

  def read(self, locs, n_bytes):
    b = []
    for f_name, offset in locs:
      if f_name not in self._open_files:
        self._open_files[f_name] = open(f_name, 'rb')
      f = self._open_files[f_name]
      f.seek(offset)
      n_read = min(n_bytes, BLOCK_SIZE - offset)
      b.append(f.read(n_read))
      n_bytes -= n_read
    return b''.join(b)
  
  def close(self):
    for f in self._open_files.values():
      f.close()

  def __exit__(self, exc_type, exc_value, traceback):
    self.close()
    return False 

In [ ]:
TUPLE_SIZE = 6       # We're going to pack the doc_id and tf values in this 
                     # many bytes.
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer

DL = {}  # We're going to update and calculate this after each document. This will be usefull for the calculation of AVGDL (utilized in BM25) 
class InvertedIndex:  
  def __init__(self, docs={}):
    """ Initializes the inverted index and add documents to it (if provided).
    Parameters:
    -----------
      docs: dict mapping doc_id to list of tokens
    """
    # stores document frequency per term
    self.df = Counter()
    # stores total frequency per term
    self.term_total = Counter()
    # stores posting list per term while building the index (internally), 
    # otherwise too big to store in memory.
    self._posting_list = defaultdict(list)
    # mapping a term to posting file locations, which is a list of 
    # (file_name, offset) pairs. Since posting lists are big we are going to
    # write them to disk and just save their location in this list. We are 
    # using the MultiFileWriter helper class to write fixed-size files and store
    # for each term/posting list its list of locations. The offset represents 
    # the number of bytes from the beginning of the file where the posting list
    # starts. 
    self.posting_locs = defaultdict(list)
    
    for doc_id, tokens in docs.items():
      self.add_doc(doc_id, tokens)

  def add_doc(self, doc_id, tokens):
    """ Adds a document to the index with a given `doc_id` and tokens. It counts
        the tf of tokens, then update the index (in memory, no storage 
        side-effects).
    """
    DL[doc_id] = DL.get(doc_id,0) + (len(tokens))
    w2cnt = Counter(tokens)
    self.term_total.update(w2cnt)
    max_value = max(w2cnt.items(), key=operator.itemgetter(1))[1]    
    # frequencies = {key: value/max_value for key, value in frequencies.items()}
    for w, cnt in w2cnt.items():        
        self.df[w] = self.df.get(w, 0) + 1                
        self._posting_list[w].append((doc_id, cnt))


  def write(self, base_dir, name):
    """ Write the in-memory index to disk and populate the `posting_locs`
        variables with information about file location and offset of posting
        lists. Results in at least two files: 
        (1) posting files `name`XXX.bin containing the posting lists.
        (2) `name`.pkl containing the global term stats (e.g. df).
    """
    #### POSTINGS ####
    self.posting_locs = defaultdict(list)
    with closing(MultiFileWriter(base_dir, name)) as writer:
      # iterate over posting lists in lexicographic order
      for w in sorted(self._posting_list.keys()):
        self._write_a_posting_list(w, writer, sort=True)
    #### GLOBAL DICTIONARIES ####
    self._write_globals(base_dir, name)

  def _write_globals(self, base_dir, name):
    with open(Path(base_dir) / f'{name}.pkl', 'wb') as f:
      pickle.dump(self, f)

  def _write_a_posting_list(self, w, writer, sort=False):
    # sort the posting list by doc_id
    pl = self._posting_list[w]
    if sort:
      pl = sorted(pl, key=itemgetter(0))
    # convert to bytes    
    b = b''.join([(int(doc_id) << 16 | (tf & TF_MASK)).to_bytes(TUPLE_SIZE, 'big')
                  for doc_id, tf in pl])
    # write to file(s)
    locs = writer.write(b)
    # save file locations to index
    self.posting_locs[w].extend(locs) 

  def __getstate__(self):
    """ Modify how the object is pickled by removing the internal posting lists
        from the object's state dictionary. 
    """
    state = self.__dict__.copy()
    del state['_posting_list']
    return state

  @staticmethod
  def read_index(base_dir, name):
    with open(Path(base_dir) / f'{name}.pkl', 'rb') as f:
      return pickle.load(f)

  @staticmethod
  def delete_index(base_dir, name):
    path_globals = Path(base_dir) / f'{name}.pkl'
    path_globals.unlink()
    for p in Path(base_dir).rglob(f'{name}_*.bin'):
      p.unlink()

In this assignment we will generate two indexes. 
One for titles and one for body (e.g., text).
First, we need to tokenize the title and body textual information (e.g., text). 
Note: it can take several minutes to tokenize it.

Next, we will create two seperated indexes.


In [ ]:
cran_txt_data_titles = {k: tokenize(v['title']) for k,v in cran_txt_data.items()}
cran_txt_data_text = {k: tokenize(v['text']) for k,v in cran_txt_data.items()}

Next, we need to preprocess the text of the queries for both training and test sets.

In [ ]:
cran_txt_query_text_train = {q[0]: tokenize(q[1]['question']) for q in cran_qry_rel_data_train}
cran_txt_query_text_test = {q[0]: tokenize(q[1]['question']) for q in cran_qry_rel_data_test}

#### Creating and writing the index

In [ ]:
index_titles = InvertedIndex(docs=cran_txt_data_titles)
index_text = InvertedIndex(docs=cran_txt_data_text)

In [ ]:
# create directories for the different indices 
!mkdir body_index title_index

In [ ]:
index_titles.write('title_index','title')
index_text.write('body_index','body')

### Reading data from posting (code taken from assignment 2)

In [ ]:
class InvertedIndex(InvertedIndex):
  
  def posting_lists_iter(self):
    """ A generator that reads one posting list from disk and yields 
        a (word:str, [(doc_id:int, tf:int), ...]) tuple.
    """
    with closing(MultiFileReader()) as reader:
      for w, locs in self.posting_locs.items():
        # read a certain number of bytes into variable b
        b = reader.read(locs, self.df[w] * TUPLE_SIZE)
        posting_list = []
        # convert the bytes read into `b` to a proper posting list.
        
        for i in range(self.df[w]):
          doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
          tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
          posting_list.append((doc_id, tf))
        
        yield w, posting_list

**Example of use**

First, we will load the title index. 

Next, we will search for a given term in both lists (words, and pls)

Remineder: pls is the information from the posting list.
Each value within the `words` list has a corresponding value in the `pls` list.
The value within a `pls` list is a list of tuples. Each tuple in the following format (x,y). Where x represent the document id, and `y` represent the occurence of the term in the document. 

In this example we are seeking the first term in the `words` list, and we can observe the is appears in 4 different documents. 
Moreover, we can observe that in document 807 it appears twice.

In [ ]:
idx_title = InvertedIndex.read_index('title_index', 'title')
idx_body = InvertedIndex.read_index('body_index', 'body')
# read posting lists from disk
words, pls = zip(*idx_title.posting_lists_iter())

Let's check for example the document number `807`. 
We needed its document length (DL) which is suppose to be the length of its title + the length of the text of the document. 
Let's verify it.

In [ ]:
#Let's check the title of document 807
cran_txt_data_titles['807']

['ground',
 'measurements',
 'shock',
 'wave',
 'noise',
 'supersonic',
 'bomber',
 'airplanes',
 'altitude',
 'range',
 '000',
 '000',
 'feet']

Clearly `000` should not appear twice.
Let have a second look on the original document and understand weather it's a mistake within a tokenization step or in the original title.

In [ ]:
cran_txt_data['807']['title']

' ground measurements of the shock wave noise from supersonic bomber airplanes in the altitude range from 30,000 to 50,000 feet . '

As we could have guessed, the token `000` does not appear twice in this document.

**Reminder: garbage in = garbage out.**

**For this assignment, it is ok to leave it as is. However, verify this part when you are working on your project.**

In [ ]:
assert DL['807'] == len(cran_txt_data_titles['807']) + len(cran_txt_data_text['807'])
assert len(DL) == len(cran_txt_data_titles)

### Ranking
In this section, you will be given a query or queries and return a ranked list of documents to retrieve.
In this assignment, we are experimenting with two methods. TF-IDF and BM25. 

**We will use the inverted index in order to do so and will not utilize the whole corpus in advance.**

#### TF-IDF for carnfield data (20 points)

Bellow cells contain the following functions: 

*   *generate_query_tfidf_vector* - Generate a vector representing the query
*   *get_candidate_documents_and_scores* - Generate a dictionary representing a pool of candidate documents for a given query.
*   *generate_document_tfidf_matrix* - Generate a DataFrame of tfidf scores for a given query.
*   *cosine_similarity* - Calculate the cosine similarity for each candidate document in D and a given query (e.g., Q). **You will impelemnt this function (10 points)**

*   *get_top_n* - Sort and return the highest N documents according to the cosine similarity score.

*   *get_topN_score_for_queries* - Generate a dictionary that gather for every query its topN score. **You will impelemnt this function (10 points)**

In [ ]:
def generate_query_tfidf_vector(query_to_search,index):
    """ 
    Generate a vector representing the query. Each entry within this vector represents a tfidf score.
    The terms representing the query will be the unique terms in the index.

    We will use tfidf on the query as well. 
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the query.    

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.    
    
    Returns:
    -----------
    vectorized query with tfidf scores
    """
    
    epsilon = .0000001
    total_vocab_size = len(index.term_total)
    Q = np.zeros((total_vocab_size))
    term_vector = list(index.term_total.keys())    
    counter = Counter(query_to_search)
    for token in np.unique(query_to_search):
        if token in index.term_total.keys(): #avoid terms that do not appear in the index.               
            tf = counter[token]/len(query_to_search) # term frequency divded by the length of the query
            df = index.df[token]            
            idf = math.log((len(DL))/(df+epsilon),10) #smoothing
            
            try:
                ind = term_vector.index(token)
                Q[ind] = tf*idf                    
            except:
                pass
    return Q

In [ ]:
def get_posting_iter(index):
    """
    This function returning the iterator working with posting list.
    
    Parameters:
    ----------
    index: inverted index    
    """
    words, pls = zip(*index.posting_lists_iter())
    return words,pls

In [ ]:
def get_candidate_documents_and_scores(query_to_search,index,words,pls):
    """
    Generate a dictionary representing a pool of candidate documents for a given query. This function will go through every token in query_to_search
    and fetch the corresponding information (e.g., term frequency, document frequency, etc.') needed to calculate TF-IDF from the posting list.
    Then it will populate the dictionary 'candidates.'
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the document.
    
    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: iterator for working with posting.
    
    Returns:
    -----------
    dictionary of candidates. In the following format:
                                                               key: pair (doc_id,term)
                                                               value: tfidf score. 
    """
    candidates = {}
    for term in np.unique(query_to_search):
        if term in words:            
            list_of_doc = pls[words.index(term)]            
            normlized_tfidf = [(doc_id,(freq/DL[str(doc_id)])*math.log(len(DL)/index.df[term],10)) for doc_id, freq in list_of_doc]
            
            for doc_id, tfidf in normlized_tfidf:
                candidates[(doc_id,term)] = candidates.get((doc_id,term), 0) + tfidf               

    return candidates

In [ ]:
def generate_document_tfidf_matrix(query_to_search,index,words,pls):
    """
    Generate a DataFrame `D` of tfidf scores for a given query. 
    Rows will be the documents candidates for a given query
    Columns will be the unique terms in the index.
    The value for a given document and term will be its tfidf score.
    
    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    
    words,pls: iterator for working with posting.

    Returns:
    -----------
    DataFrame of tfidf scores.
    """
    
    total_vocab_size = len(index.term_total)
    candidates_scores = get_candidate_documents_and_scores(query_to_search,index,words,pls) #We do not need to utilize all document. Only the docuemnts which have corrspoinding terms with the query.
    unique_candidates = np.unique([doc_id for doc_id, freq in candidates_scores.keys()])
    D = np.zeros((len(unique_candidates), total_vocab_size))
    D = pd.DataFrame(D)
    
    D.index = unique_candidates
    D.columns = index.term_total.keys()

    for key in candidates_scores:
        tfidf = candidates_scores[key]
        doc_id, term = key    
        D.loc[doc_id][term] = tfidf

    return D

**YOUR TASK (10 POINTS):** Complete the implementation of `cosine_similarity`. This function calculate the cosine similarity for each candidate document in D and a given query (e.g., Q) and return a dictionary of cosine similary scores.

**Note:** for this task you cannot use sklearn. However, you can use pandas or numpy.

In [ ]:
def cosine_similarity(D,Q):
    """
    Calculate the cosine similarity for each candidate document in D and a given query (e.g., Q).
    Generate a dictionary of cosine similarity scores 
    key: doc_id
    value: cosine similarity score
    
    Parameters:
    -----------
    D: DataFrame of tfidf scores.

    Q: vectorized query with tfidf scores
    
    Returns:
    -----------
    dictionary of cosine similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: cosine similarty score.
    """
    fin_dict = {}
    normQ = np.linalg.norm(Q)
    for doc_id in D.rows.index:
      A = D.loc[doc_id]
      normA = np.linalg.norm(A)
      cosine = np.dot(A,Q)/(normA*normQ)
      fin_dict[doc_id] = cosine
    
    return fin_dict
########################################################################################################################

In [ ]:
def get_top_n(sim_dict,N=3):
    """ 
    Sort and return the highest N documents according to the cosine similarity score.
    Generate a dictionary of cosine similarity scores 
   
    Parameters:
    -----------
    sim_dict: a dictionary of similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: similarity score. We keep up to 5 digits after the decimal point. (e.g., round(score,5))

    N: Integer (how many documents to retrieve). By default N = 3
    
    Returns:
    -----------
    a ranked list of pairs (doc_id, score) in the length of N.
    """
    
    return sorted([(doc_id,round(score,5)) for doc_id, score in sim_dict.items()], key = lambda x: x[1],reverse=True)[:N]

**YOUR TASK (10 POINTS)**: Complete the implementation of `get_topN_score_for_queries`. This function generate a dictionary that gather for every query its topN score, based on cosine similarity.

In [ ]:
def get_topN_score_for_queries(queries_to_search,index,N=3):
    """ 
    Generate a dictionary that gathers for every query its topN score.
    
    Parameters:
    -----------
    queries_to_search: a dictionary of queries as follows: 
                                                        key: query_id
                                                        value: list of tokens.
    index:           inverted index loaded from the corresponding files.    
    N: Integer. How many documents to retrieve. This argument is passed to the topN function. By default N = 3, for the topN function. 
    
    Returns:
    -----------
    return: a dictionary of queries and topN pairs as follows:
                                                        key: query_id
                                                        value: list of pairs in the following format:(doc_id, score). 
    """
    ################################################################################################################################################
    # for query_id in queries_to_search.keys():
    words, pls = get_posting_iter(index)
    fin_dict = {}
    for id, query in queries_to_search.items():
      D = generate_document_tfidf_matrix(query,index,words,pls)
      Q = generate_query_tfidf_vector(query,index)

      score_dict = cosine_similarity(D,Q)
      fin_dict[id] = get_top_n(score_dict,N)
    
    return fin_dict

In [ ]:
tfidf_queries_score_train = get_topN_score_for_queries(cran_txt_query_text_train,idx_title)

{12: 0.3612568655672294, 13: 0.7072684769411193, 47: 0.285247189036838, 51: 0.439888117996422, 52: 0.3612568655672294, 56: 0.3696218986930113, 57: 0.3696218986930113, 66: 0.3791808217448805, 78: 0.28524718903683804, 100: 0.28524718903683804, 102: 0.33486653684866236, 141: 0.36125686556722947, 154: 0.3791808217448805, 184: 0.3348665368486623, 193: 0.23426426486372676, 197: 0.27500323112159414, 202: 0.28524718903683804, 204: 0.3612568655672294, 209: 0.28524718903683804, 211: 0.27500323112159414, 215: 0.23426426486372676, 244: 0.2342642648637268, 248: 0.27500323112159414, 252: 0.3348665368486623, 253: 0.28524718903683804, 264: 0.23426426486372676, 280: 0.3612568655672294, 285: 0.23426426486372678, 302: 0.23426426486372678, 309: 0.36125686556722947, 311: 0.3348665368486623, 315: 0.2342642648637268, 316: 0.23426426486372678, 327: 0.3696218986930113, 349: 0.3696218986930113, 359: 0.3696218986930113, 373: 0.2342642648637268, 378: 0.2342642648637268, 391: 0.23426426486372678, 413: 0.2342642648

In [ ]:
#tests 
assert len(tfidf_queries_score_train)==180
assert 0 not in tfidf_queries_score_train.keys()
assert len(tfidf_queries_score_train[5])==3
assert tfidf_queries_score_train[172][0][1] == tfidf_queries_score_train[172][1][1]
assert tfidf_queries_score_train[14][0][1] == tfidf_queries_score_train[172][1][1]

For query 172 we can observe two document with cosine similarity score of 1. Let's have a glance on this query and documents for making sure it makes sense

In [ ]:
print('relevnt documents and tfidf score for query number 172 :',tfidf_queries_score_train[172])
print('query: ' ,cran_txt_query_text_train[172])
print('docuemnt 320: ', cran_txt_data_titles['320'])
print('docuemnt 320: ', cran_txt_data_titles['321'])
print('docuemnt 322: ', cran_txt_data_titles['322'])

relevnt documents and tfidf score for query number 172 : [(320, 1.0), (321, 1.0), (322, 1.0)]
query:  ['solution', 'blasius', 'problem', 'three-point', 'boundary', 'conditions']
docuemnt 320:  ['comment', 'improved', 'numerical', 'solution', 'blasius', 'problem', 'three-point', 'boundary', 'conditions']
docuemnt 320:  ['improved', 'numerical', 'solution', 'blasius', 'problem', 'three-point', 'boundary', 'conditions']
docuemnt 322:  ['numerical', 'solution', 'blasius', 'problem', 'three-point', 'boundary', 'conditions']


#### BM25 for carnfield data (15 points)
As a reminder:

To use BM25 we will need to following parameters:

* $k1$ and $b$ - free parameters
* $f(t_i,D)$ - term frequency of term $t_i$ in document $D$
* |$D$|- is the length of the document $D$ in terms 
* $avgdl$ -  average document length
* $IDF$ - which is calculted as follows: $ln(\frac{(N-n(t_i)+0.5}{n(t_i)+0.5)}+1)$, where $N$ is the total number of documents in the collection, and $n(t_i)$ is the number of documents containing $t_i$ (e.g., document frequency (df)).

Now, we will use the inverted index to fetch this information.

**We need to check only documents which are 'candidates' for a given query.**

**YOUR TASK (15 POINTS):** Complete the implementation of `search` at the BM25 class.
Differently, from previous implememntation, this time you should check only documents which are 'candidates' for a given query. 


In [ ]:
import math
from itertools import chain
import time
# When preprocessing the data have a dictionary of document length for each document saved in a variable called `DL`.
class BM25_from_index:
    """
    Best Match 25.    
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    index: inverted index
    """

    def __init__(self,index,k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1
        self.index = index
        self.N = len(DL)
        self.AVGDL = sum(DL.values())/self.N
        self.words, self.pls = zip(*self.index.posting_lists_iter())        

    def calc_idf(self,list_of_tokens):
        """
        This function calculate the idf values according to the BM25 idf formula for each term in the query.
        
        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        
        Returns:
        -----------
        idf: dictionary of idf scores. As follows: 
                                                    key: term
                                                    value: bm25 idf score
        """        
        idf = {}        
        for term in list_of_tokens:            
            if term in self.index.df.keys():
                n_ti = self.index.df[term]
                idf[term] = math.log(1 + (self.N - n_ti + 0.5) / (n_ti + 0.5))
            else:
                pass                             
        return idf
        

    def search(self, queries,N=3):
        """
        This function calculate the bm25 score for given query and document.
        We need to check only documents which are 'candidates' for a given query. 
        This function return a dictionary of scores as the following:
                                                                    key: query_id
                                                                    value: a ranked list of pairs (doc_id, score) in the length of N.
        
        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.
        
        Returns:
        -----------
        score: float, bm25 score.
        """
        # aBOVE EXPLANATION IS SHIT
        """
        This function use the _score function to calculate the bm25 score for all queries provided.
        
        Parameters:
        -----------
        queries: list of lists. Each inner list is a list of tokens. For example:
                                                                                    [
                                                                                        ['look', 'blue', 'sky'],
                                                                                        ['likes', 'blue', 'sun'],
                                                                                        ['likes', 'diamonds']
                                                                                    ]

        Returns:
        -----------
        list of scores of bm25
        """
        #########################################################################################
        words, pls = get_posting_iter(self.index)
        candidates = {}
        
        for id, query in queries.items():
          doc_list = []
          doc_dict = {}
          for term in query:
            if term in words:          
              chosen = words.index(term)
              for tup in pls[chosen]:
                if tup[0] in doc_dict:
                  continue
                else:
                  doc_list.append((tup[0],self._score(query,tup[0])))
                  doc_dict[tup[0]] = 1
          doc_list = sorted(doc_list, key = lambda x: x[1], reverse = True)
          if N<=len(doc_list):
            candidates[id] = doc_list[:N]
          else:
            candidates[id] = doc_list
        return candidates
        ##############################################################################################

    def _score(self, query, doc_id):
        """
        This function calculate the bm25 score for given query and document.
        
        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.
        
        Returns:
        -----------
        score: float, bm25 score.
        """        
        score = 0.0        
        doc_len = DL[str(doc_id)]        
        self.idf = self.calc_idf(query)
        for term in query:
            if term in self.index.term_total.keys():                
                term_frequencies = dict(self.pls[self.words.index(term)])                
                if doc_id in term_frequencies.keys():            
                    freq = term_frequencies[doc_id]
                    numerator = self.idf[term] * freq * (self.k1 + 1)
                    denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.AVGDL)
                    score += (numerator / denominator)
        return score

In [ ]:
bm25_title = BM25_from_index(idx_title)
bm25_queries_score_train = bm25_title.search(cran_txt_query_text_train)

In [ ]:
#tests
assert len(bm25_queries_score_train)==180
assert 0 not in bm25_queries_score_train.keys()
assert len(bm25_queries_score_train[5])==3
assert bm25_queries_score_train[172][0][1] != bm25_queries_score_train[172][1][1]
assert bm25_queries_score_train[14][0][1] != bm25_queries_score_train[172][1][1]

## Task 3: Using weights of title and body scores (25 points)

Reminder: we are building on the training set.
Later you need to optimize the parameters and run the best parameters on the test set.

Now we will experiment with two sets of results. 
The first corresponds to results from the title index. 
The second corresponds to the results from the body index.

We need to merge them into a single result set.

**YOUR TASK (15 POINTS):** Complete the implementation of `merge_results`.
This function merge and sort documents retrieved by its weighte score (e.g., title and body).


In [ ]:
def merge_results(title_scores,body_scores,title_weight=0.5,text_weight=0.5,N = 3):    
    """
    This function merge and sort documents retrieved by its weighte score (e.g., title and body). 

    Parameters:
    -----------
    title_scores: a dictionary build upon the title index of queries and tuples representing scores as follows: 
                                                                            key: query_id
                                                                            value: list of pairs in the following format:(doc_id,score)
                
    body_scores: a dictionary build upon the body/text index of queries and tuples representing scores as follows: 
                                                                            key: query_id
                                                                            value: list of pairs in the following format:(doc_id,score)
    title_weight: float, for weigted average utilizing title and body scores
    text_weight: float, for weigted average utilizing title and body scores
    N: Integer. How many document to retrieve. This argument is passed to topN function. By default N = 3, for the topN function. 
    
    Returns:
    -----------
    dictionary of querires and topN pairs as follows:
                                                        key: query_id
                                                        value: list of pairs in the following format:(doc_id,score). 
    """

    #No idea what to do 
    fin_dict = {}
    for qe_id, tit in title_scores.items():
      bod_list = body_scores[qe_id]
      query_counter = Counter()
      for tit_p in tit:
        query_counter[tit_p[0]] += tit_p[1]*title_weight
      for bod_p in bod_list:
        query_counter[bod_p[0]] += bod_p[1]*text_weight
      top_N_in_counter = query_counter.most_common(N)
      fin_dict[qe_id] = top_N_in_counter    
    return fin_dict

In [ ]:
bm25_title = BM25_from_index(idx_title)
bm25_body = BM25_from_index(idx_body)

bm25_queries_score_train_title = bm25_title.search(cran_txt_query_text_train)
bm25_queries_score_train_body = bm25_body.search(cran_txt_query_text_train)

In [ ]:
#tests
w1,w2 = 0.5, 0.5
w3,w4 = 0.25,0.75

half_and_half = merge_results(bm25_queries_score_train_title,bm25_queries_score_train_body,w1,w2)  
assert len(half_and_half[2]) == 3
assert type(half_and_half) == dict
assert type(half_and_half[2]) == list
assert len(half_and_half) == 180
assert half_and_half[2][0][1] == 0.5 * (bm25_queries_score_train_title[2][-1][1]+ bm25_queries_score_train_body[2][0][1])

quarter_and_three_quarters = merge_results(bm25_queries_score_train_title,bm25_queries_score_train_body,0.25,0.75)        

assert quarter_and_three_quarters[2][0][1] == (w3 * bm25_queries_score_train_title[2][-1][1] + w4 * bm25_queries_score_train_body[2][0][1])
assert {k for k,v in half_and_half[16]} != {k for k,v in quarter_and_three_quarters[16]}
assert len({k for k,v in half_and_half[16]}.union({k for k,v in quarter_and_three_quarters[16]})) < (len({k for k,v in half_and_half[16]}) + len({k for k,v in quarter_and_three_quarters[16]}))

{1: [(13, 18.282039867657108), (875, 15.255061987857486), (746, 14.001624625702807)], 2: [(746, 23.172761153857433), (700, 18.53240582029468), (12, 17.777104847031126)], 3: [(399, 26.899409768681274), (181, 16.614393161640493), (144, 14.575632984250907)], 4: [(166, 22.632717112187606), (488, 16.54751069532328), (1312, 14.956722071229022)], 5: [(708, 7.854658806176595), (1102, 7.668224422874614), (1272, 6.864713570540218)], 6: [(385, 10.979021377437057), (544, 10.505412066109754), (271, 10.247676171495687)], 7: [(248, 31.696401568953494), (57, 28.116174633987317), (469, 27.247997494539838)], 8: [(711, 18.814016873821622), (461, 16.692991184545917), (1311, 13.856052305689772)], 9: [(21, 16.975330462804642), (5, 11.423040950547787), (326, 9.780797660686526)], 10: [(691, 15.67906502317173), (302, 14.744893570950413), (621, 11.938056933122166)], 11: [(495, 19.11941407782248), (320, 12.456381527020742), (654, 11.806490927553346)], 12: [(624, 15.59029206974229), (1334, 10.080470928879329), (6

**YOUR TASK (10 POINTS):** provide three examples of mistakes that the model is making and explanations for why, and describe how you will change the model based on these observations.

In [ ]:
# Examples of mistakes 
#1. The model fails to account for relevance of words that occur frequently in the document.
#2. The model is not able to account for the context of words, as it only considers the frequency of words.
#3. The model is unable to distinguish between different types of documents, such as text, images, and videos.

# Explain
#1. The BM25 model fails to account for the relevance of words that occur frequently in the document. 
#   This is because the model simply counts the number of times a word appears in the document, without taking into account its relevance or context.
#2. The BM25 model is not able to capture the context of words.
#   This is because the model simply counts the number of times a word appears in the document, without taking into consideration the relationship between words or the context of the sentence.
#3. The BM25 model is unable to distinguish between different types of documents, such as text, images, and videos.
#   This is because the model only considers the frequency of words, without taking into consideration the type of document.

# How to fix
#1. To fix the issue with frequency of words, the BM25 model can be adjusted to utilize a tf-idf weighting scheme instead of a simple term frequency count.
#2. To address the issue of context, the model can be adjusted to incorporate Word2Vec into the scoring process.
#3. To address the issue of different types of documents, the model can be adjusted to utilize a CNN to recognize the type of document and to adjust the scoring accordingly.